In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/finance-data/finance.index
/kaggle/input/finance-data/chunks.jsonl


In [6]:
pip install pdfplumber sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 66.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 79.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.2 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB

In [ ]:
pip install langchain_community

In [7]:
from datasets import load_dataset

ds = load_dataset("PatronusAI/financebench")


README.md: 0.00B [00:00, ?B/s]

financebench_merged.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/150 [00:00<?, ? examples/s]

In [8]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['financebench_id', 'company', 'doc_name', 'question_type', 'question_reasoning', 'domain_question_num', 'question', 'answer', 'justification', 'dataset_subset_label', 'evidence', 'gics_sector', 'doc_type', 'doc_period', 'doc_link'],
        num_rows: 150
    })
})


In [9]:
df = ds["train"].to_pandas()
df=df[:10]
print(df["question"][0]+df["justification"][0])

What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.The metric capital expenditures was directly extracted from the company 10K. The line item name, as seen in the 10K, was: Purchases of property, plant and equipment (PP&E).


In [10]:
import os
import requests
import pdfplumber
import json
from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import base64

2025-10-08 15:09:00.164365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759936140.427454      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759936140.492604      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
def download_pdf(url: str, save_dir="pdfs") -> str:
    print("Downloading pdf")
    os.makedirs(save_dir, exist_ok=True)
    filename = os.path.join(save_dir, url.split("/")[-1])
    if not os.path.exists(filename):
        r = requests.get(url)
        with open(filename, "wb") as f:
            f.write(r.content)
    return filename

In [12]:
def extract_pdf_content(pdf_path: str) -> Dict:
    print("Extracting pdf content")
    content = {"text": [], "tables": [], "figures": []}

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            # Text
            text = page.extract_text()
            if text:
                content["text"].append({"page": page_num, "content": text})

            # Tables
            tables = page.extract_tables()
            for t_id, table in enumerate(tables):
                table_str = "\n".join([", ".join([cell or "" for cell in row]) for row in table])
                content["tables"].append({
                    "page": page_num,
                    "table_id": t_id,
                    "content": table_str
                })

            # Figures (basic: extract image bytes)
            images = page.images
            for f_id, img in enumerate(images):
                x0, top, x1, bottom = img["x0"], img["top"], img["x1"], img["bottom"]
                # crop the image region
                cropped = page.within_bbox((x0, top, x1, bottom)).to_image(resolution=150)
                img_bytes = cropped.original.save("tmp.png")  # hack, can be improved
                with open("tmp.png", "rb") as imf:
                    b64 = base64.b64encode(imf.read()).decode("utf-8")
                content["figures"].append({
                    "page": page_num,
                    "figure_id": f_id,
                    "content": b64  # stored as base64
                })

    return content

In [13]:
def chunk_content(content: Dict, chunk_size=1000, chunk_overlap=100) -> List[Dict]:
    print("Creating Chunks")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks = []

    # Text → split
    for t in content["text"]:
        split_texts = splitter.split_text(t["content"])
        for j, c in enumerate(split_texts):
            chunks.append({
                "content": c,
                "modality": "text",
                "page": t["page"]
            })

    # Tables → keep whole
    for tbl in content["tables"]:
        chunks.append({
            "content": tbl["content"],
            "modality": "table",
            "page": tbl["page"]
        })

    # Figures → keep as single chunk (store base64 reference or path)
    for fig in content["figures"]:
        chunks.append({
            "content": fig["content"],
            "modality": "figure",
            "page": fig["page"]
        })

    return chunks

In [14]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

def generate_chunk_heading(llm, chunk: str) -> str:
    
    # Define prompt
    prompt = PromptTemplate(
        input_variables=["chunk"],
        template=(
            "You are given a piece of text from a financial document.\n"
            "Generate a very concise heading (maximum 10 words) "
            "that best summarizes it.\n\n"
            "Text:\n{chunk}\n\n"
            "Heading:"
        )
    )

   
    chain = LLMChain(llm=llm, prompt=prompt)

    
    heading = chain.run({"chunk": chunk}).strip()

    enriched_chunk = f"Heading: {heading} | Content: {chunk}"
    return enriched_chunk

In [15]:
import pytesseract
from PIL import Image
import io, base64

def extract_text_from_image(base64_image) -> str:
    image_data = base64.b64decode(base64_image)
    image = Image.open(io.BytesIO(image_data))
    text = pytesseract.image_to_string(image)
    return text.strip()


In [16]:
def embed_chunks(chunks: List[Dict], model) -> List[Dict]:
    # For now: treat text, table, figure uniformly as text
    # Figures (base64) → could later use CLIP for multimodal embeddings
    print("Generating embeddings")
    contents = []
    for ch in chunks:
        if ch["modality"] == "figure":
            content=extract_text_from_image(ch["content"])
            contents.append(content)
        else:
            contents.append(ch["content"])

    vectors = model.encode(contents, convert_to_numpy=True)

    # attach embeddings back
    for i, emb in enumerate(vectors):
        chunks[i]["embedding"] = emb
    return chunks


In [18]:
import os, json, faiss, numpy as np
def build_faiss_index(
    chunks: List[Dict], dim: int,
    faiss_path="/kaggle/working/finance.index", # File path for saving
    meta_path="/kaggle/working/chunks.jsonl"
):
    print("Storing in faiss")

    index = faiss.IndexFlatL2(dim)
    embeddings = [c["embedding"] for c in chunks]
    index.add(np.array(embeddings, dtype="float32"))

    # Save FAISS index
    faiss.write_index(index, faiss_path)

    # Save metadata
    with open(meta_path, "w", encoding="utf-8") as f:
        for c in chunks:
            meta = {k: v for k, v in c.items() if k != "embedding"}
            f.write(json.dumps(meta) + "\n")

    return index

In [20]:
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline


hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base"
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


def pipeline(doc_links: List[str]):
    "Initiating pipeline"
    model = SentenceTransformer("intfloat/multilingual-e5-large")
    all_chunks = []

    for url in doc_links:
        pdf_path = download_pdf(url)
        content = extract_pdf_content(pdf_path)
        chunks = chunk_content(content)       
        em_chunks = embed_chunks(chunks, model)
        all_chunks.extend(em_chunks)

    print(f"Finished pipeline. Stored {len(all_chunks)} chunks embeddings")
    return all_chunks

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
/tmp/ipykernel_36/113847593.py:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [24]:
a=set()
docs=[]
for i in range(len(df["doc_name"])):
    if df["doc_name"][i] not in a:
        docs.append(df["doc_link"][i])
        a.add(df["doc_name"][i])
print(f"Total number of docs to be processed: {len(docs)}")
allchunks=pipeline(docs)

Total number of docs to be processed: 4


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

Extracting pdf content
Creating Chunks
Generating embeddings


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Extracting pdf content
Creating Chunks
Generating embeddings


Batches:   0%|          | 0/37 [00:00<?, ?it/s]

Finished pipeline. Stored 4270 chunks embeddings


In [25]:
dim = len(allchunks[0]["embedding"])
build_faiss_index(allchunks, dim)

Storing in faiss


<faiss.swigfaiss_avx512.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7ded9fd20720> >

In [29]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
import faiss, json

# write faiss index, metadata file loction
index = faiss.read_index("/kaggle/working/finance.index") #Here 

with open("/kaggle/working/chunks.jsonl", "r", encoding="utf-8") as f: #Here Also
    chunks = [json.loads(line) for line in f]

documents = [Document(page_content=c["content"], metadata=c) for c in chunks]

docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})
index_to_docstore_id = {i: str(i) for i in range(len(documents))}
model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
vectorstore = FAISS(
    model,   
    index,             
    docstore,
    index_to_docstore_id
)
q="What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.The metric capital expenditures was directly extracted from the company 10K. The line item name, as seen in the 10K, was: Purchases of property, plant and equipment (PP&E)."
results = vectorstore.similarity_search(q, k=10)

for r in results:
    print(r.page_content[:200], r.metadata)


/tmp/ipykernel_36/955197057.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")


Major GAAP Cash Flow Categories
Net cash provided by operating activities $ 6,439 $ 6,240 $ 6,662
Net cash provided by (used in) investing activities 222 (3,086) (1,403)
Net cash used in financing act {'content': 'Major GAAP Cash Flow Categories\nNet cash provided by operating activities $ 6,439 $ 6,240 $ 6,662\nNet cash provided by (used in) investing activities 222 (3,086) (1,403)\nNet cash used in financing activities (6,701) (2,655) (4,626)\nFree Cash Flow (non-GAAP measure)\nNet cash provided by operating activities $ 6,439 $ 6,240 $ 6,662\nPurchases of property, plant and equipment (PP&E) (1,577) (1,373) (1,420)\nFree cash flow $ 4,862 $ 4,867 $ 5,242\nNet income attributable to 3M $ 5,349 $ 4,858 $ 5,050\nFree cash flow conversion 91% 100% 104%\nOff-Balance Sheet Arrangements and Contractual Obligations:\nAs of December 31, 2018, the Company has not utilized special purpose entities to facilitate off-balance sheet financing arrangements. Refer to the section\nentitled “Warrantie